In [80]:
import pandas as pd
import numpy as np
import pickle as p

In [81]:
data_all = pd.read_csv('../data_all.csv')

# Test with Subset of Data
np.random.seed(43)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=153, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()

# # Test with All Data
# data_subset = data_all.copy()

In [82]:
# item_subset

### `/train`

In [83]:
import time

In [84]:
import requests
import json
from threading import Thread

url = 'https://fillet.azurewebsites.net/train/'
# url = 'http://localhost:5000/train/'


data_subset.to_parquet('data_subset.parquet')
files = {'data': open('data_subset.parquet', 'rb')}

payload = {'cv_acc': True, 'project_id': 1}

Thread(target=requests.post, args=(url, ), kwargs={'data': payload, 'files': files}).start()

time.sleep(10)
files['data'].close()

### `/query_progress`

In [111]:
import requests
import json

url = 'https://fillet.azurewebsites.net/query_progress/'
# url = 'http://localhost:5000/query_progress/'


payload = {'project_id': 1}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=5)

print('% Complete:',round(r.json()['pct_complete'],3))
print('CV % Complete:',round(r.json()['cv_progress'],3))
print('CV Done:',r.json()['cv_done'])

% Complete: 100.0
CV % Complete: 19.6
CV Done: 0


In [110]:
set(r.json()['project_items']) - set(r.json()['train_complete'])

set()

### `/get_cv_results`

In [105]:
import requests
import json

url = 'https://fillet.azurewebsites.net/get_cv_results/'
# url = 'http://localhost:5000/get_cv_results/'


payload = {'project_id': 1}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=5)
pd.DataFrame().from_dict((json.loads(r.json())))

,item_id,avg_sales,r2_score,mae_score,mpe_score,rmse_score
0,1002.0,2610.850342,0.714290,428.208115,0.164011,522.476949
1,102.0,178.524384,0.677324,32.323790,0.181061,50.155280
2,1152.0,113.752693,0.623570,22.774421,0.200210,28.569260
3,1153.0,281.756165,0.497398,81.331184,0.288658,100.619518
4,1195.0,130.380447,0.677700,26.961260,0.206789,35.028395
5,11.0,218.526474,0.875610,24.577284,0.112468,35.773224
6,1203.0,199.803940,0.852295,23.710330,0.118668,32.140583
7,1204.0,292.541565,0.885296,32.141876,0.109871,46.569782
8,1205.0,180.181778,0.828786,25.220660,0.139973,33.869499
9,1221.0,246.273209,0.768555,57.330719,0.232793,80.759911


In [127]:
r.json()

{'status': 'incomplete'}

### `/optimize`

In [ ]:
url = 'http://localhost:5000/optimize/'

payload = {'project_id': 1,
           'constraints': [],
           'population': 100,
           'max_epoch': 200
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [ ]:
best_prices = r.json()['result']
best_prices

In [ ]:
import requests
import json

url = 'http://localhost:5000/predict/'
payload = {'prices': best_prices,
           'project_id': 1
           }
headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }
payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

# What If Prediction
qty = r.json()['qty_estimates']

In [ ]:
qty

In [ ]:
# Calculate Revenue
revenue = 0
for item in best_prices.keys():
    revenue += (best_prices[item]*qty['Qty_'+item])
    
print(revenue)

### `/predict/`

In [60]:
# Sample Actual Price 
from helper_functions import *

sales_data = data_subset

sales_data = optimize_memory(sales_data)

sales_data_wide = sales_data.set_index(
    ['Wk', 'Tier', 'Store', 'Item_ID']).unstack(level=-1).reset_index().copy()
sales_data_wide.columns = [
    ''.join(str(i) for i in col).strip()
    for col in sales_data_wide.columns.values
]
sales_data_wide = sales_data_wide.sort_values(
    ['Tier', 'Store', 'Wk'], ascending=True).reset_index(drop=True)

sales_data_wide_clean = sales_data_wide.dropna(axis=0).copy()
dataset = sales_data_wide_clean
price_columns = [
    col for col in sales_data_wide_clean.columns if col.startswith('Price')
]

a = dataset.sample(1)
a_dict = a[price_columns].iloc[0].to_dict()
a_input = {}
for key in a_dict.keys():
    a_input[key.split('_')[1]]=round(a_dict[key],2)

print('Input Prices Quantities:')
# a_input


# Predict Sales Qty and Compare to Actual

import requests
import json

url = 'https://fillet.azurewebsites.net/predict/'
# url = 'http://localhost:5000/predict/'

payload = {'prices': a_input, 'project_id': 1}

headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)
qty = r.json()['qty_estimates']
qty_outp = {}
for key in qty.keys():
    qty_outp[key] = round(qty[key], 0)
actual = a[[
    col for col in sales_data_wide_clean.columns if col.startswith('Qty_')
]].iloc[0].to_dict()
pd.DataFrame([qty_outp, actual], index=['Estimated', 'Actual']).transpose()

Input Prices Quantities:


,Estimated,Actual
Qty_1004,4010.0,3189.0
Qty_102,249.0,183.0
Qty_11,273.0,229.0
Qty_1152,115.0,101.0
Qty_1195,129.0,119.0
...,...,...
Qty_7434,206.0,148.0
Qty_7575,107.0,69.0
Qty_7965,46.0,51.0
Qty_7966,146.0,160.0
